![AA](https://i0.wp.com/neptune.ai/wp-content/uploads/2022/10/Autoencoders-graph.png?resize=768%2C576&ssl=1)
____

<font size=+3 color=#AA55FF> ICA: Tensorflow and Autoencoders </font>

____

It's almost spring break!

In this ICA we have two goals:
* learn a bit about the `Tensorflow` (TF) library
* learn how to do dimensionality reduction with an autoencoder.

All of the great libraries in `sklearn` are always there for you to use: there is no reason to not use `sklearn` and `Tensorflow` together. As `Tensorflow` is more concerned with deep learning, `sklearn` offers data preprocessing and other ML estimators not in `Tensorflow`. 

As you work through this ICA, think about whether your project could use deep learning or high performance computing (e.g., use of [GPUs](https://www.tensorflow.org/guide/gpu)). 

We will use a familiar dataset today, the MNIST handwritten digits, so that you don't need to learn a new dataset and the visualizations are easy to interpret. 

____

<font size=+1 color=#AA55FF> Structure: Classification and Dimensionality Reduction </font>

____

First, I am going to give you some TF code that does classification of the MNIST data. This will familiarize you with the steps TF uses and what some of the options are. Then, you will code the autoencoder (AE) portion, with less guidance.

The first thing you need to do is ensure everyone in your group has TF installed. Take a moment to ensure that is true, and test it by running the next code cell below.

____

<font size=+1 color=#FF55AA> Classification  </font>

____

In [4]:
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.datasets import mnist
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Dense, Flatten, Reshape
from tensorflow.keras.utils import to_categorical

____

<font color=#FF55AA> Data Preparation  </font>
____

Run the code below and comment on what it does. For example, what datatypes are being used? How do you get one input image and display it? What is the `to_categorical` doing here? 

In [5]:

# load MNIST data
(train_images, train_labels), (test_images, test_labels) = mnist.load_data()

# scale the images to [0,1]
train_images = train_images / 255.0
test_images = test_images / 255.0

# one-hot encoding
train_labels = to_categorical(train_labels)
test_labels = to_categorical(test_labels)

____

<font color=#FF55AA> Build TF Model  </font>
____

Examine this syntax to understand the logic/API of how you specify a NN to TF. Describe in your own words what each step is doing: `Sequential`, `Flatten`, etc. Use the [online docs](https://www.tensorflow.org/guide/keras/sequential_model) to see other ways to do the same thing, such as using `.add`. What other activation functions are available? Why is softmax used here? 


In [6]:

# simple feedforward neural network model
model = Sequential([
    Flatten(input_shape=(28, 28)),  # input layer: Flatten the 28x28 images
    Dense(128, activation='relu'),  # hidden layer with 128 neurons and ReLU activation
    Dense(10, activation='softmax')  # output layer with 10 neurons (for 10 digits) and softmax activation
])


2024-02-21 13:12:48.206669: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:268] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected


____

<font color=#FF55AA> Training the NN  </font>
____

With the NN architecture specified, we can train it. Discuss the code below with your group and describe what it does. Vary the parameters to see how they impact the accuracy. 


In [ ]:
# compile the model
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# train/fit the model
model.fit(train_images, train_labels, epochs=5, batch_size=32)

# evaluate the model on test data
test_loss, test_acc = model.evaluate(test_images, test_labels)

print(f'Test accuracy: {test_acc}')

____

<font color=#FF55AA> Classification Summary  </font>
____

You have started your career in deep learning - congrats! 🥳

As you can see, the TF API is very well designed. In fact, the true TF API is not so easy because TF is a very low-level library (think of programming in machine language). It is the `Keras` wrapper that makes TF so easy for us to use, which is why you see that name in the `import` statements. 

If you plan to do classification in your project, especially if your dataset is very large, consider using TF. You can, and should, compare with many other estimators, so you can also include several estimators from `sklearn`, if that makes sense for you. You have some starter code 👆🏻 up there and there are many places you can take it. 

____

<font size=+1 color=#FF55AA> Dimensionality Reduction  </font>
____

The classification problem you just completed is fairly standard in two ways: (1) there are many ways to do classification and that NN is just one particular variant, and (2) the NN itself is a feed forward (sequential) architecture, perhaps the simplest one.

To fully grasp the wide applicability of NNs in general, we will use TF to build a NN that does something completely different: dimensionality reduction in an unsupervised setting. 

The architecture we will use is called an <font color=#0055FF>autoencoder</font> (AE), which attempts to produce the same information on its input and output; that is, it "does nothing". 

The key idea is the low-dimensional latent space, which is what you will vary.

____

<font color=#FF55AA> Build AE  </font>
____

Let's start with some preprocessing. Look at the figure at the top of this notebook; this is roughly what we want to build. To make the connection to the figure clearer, we will flatten the data first. Confirm that this works.

In [ ]:
# flatten the images
train_images = train_images.reshape((len(train_images), -1))
test_images = test_images.reshape((len(test_images), -1))


Next, we need to specific how many dimensions to go down to, and we'll want to vary this. 

In [ ]:
encoding_dim = 8

We will use the [`Model` library](https://keras.io/api/models/model/) (read the docs in this link) in TF to build the AE, so that you can see how this works. As you will see, this allows you to build very diverse NNs. 

Examine this code, discuss it and write your comments on how it works. Note that it adds new layers in a slightly different way: it creates a layer and passes that as the input to a function that creates the next layer, then repeats until done. You might want to draw a picture of what each step is doing. 

In [ ]:
# encoder model
encoder_input = tf.keras.Input(shape=(784,))
encoded = Dense(128, activation='relu')(encoder_input)
encoded = Dense(encoding_dim, activation='relu')(encoded)

# decoder model
decoded = Dense(128, activation='relu')(encoded)
decoded_output = Dense(784, activation='sigmoid')(decoded)

# AE
autoencoder = Model(encoder_input, decoded_output)

The reason for doing it this way is that you will often want to use just the encoder or decoder parts separately after training. Remember, the overall goal of the AE is to "do nothing"! If we didn't want access to the latent space and just wanted to train an AE, we could use this code. Don't uncomment it so that it doesn't interfere with the other code - I just want you to see an alternate way to use the TF API.

In [ ]:
# input_img = Input(shape=(784,))
# encoded = Dense(128, activation='relu')(input_img)
# encoded = Dense(32, activation='relu')(encoded)  # 32-dimensional encoded representation
# decoded = Dense(128, activation='relu')(encoded)
# decoded = Dense(784, activation='sigmoid')(decoded)

Ok, let's build _just_ the encoder and _just_ the decoder; we'll use this to probe the AE and make images below.

In [ ]:
# just encoder: only to the ltent space
encoder = Model(encoder_input, encoded)

# just decoder: starts from the latent space
encoded_input = tf.keras.Input(shape=(encoding_dim,))
decoder_layer1 = autoencoder.layers[-2]
decoder_layer2 = autoencoder.layers[-1]
decoder = Model(encoded_input, decoder_layer2(decoder_layer1(encoded_input)))

____

<font color=#FF55AA> Train AE  </font>
____


In [ ]:
autoencoder.compile(optimizer='adam', loss='binary_crossentropy')

# Train the autoencoder
autoencoder.fit(train_images, train_images,
                epochs=5,
                batch_size=256,
                shuffle=True,
                validation_data=(test_images, test_images))

____

<font color=#FF55AA> Confusion!!  </font>
____

The code below allows you to visualize what the AE is doing. Here is the logic of the code:
* two images are selected
* these two images are placed in the low-dimensionality latent space
* the images are _interpolated in the latent space_, creating a continuous new image there
* that new image is then decoded to see what the AE thinks it is! (color images are originals)
* play with the code, be sure you understand it, and be sure to vary the dimensions in the latent space! can you confuse the AE by going to small??

Note that if the dimensionality is too low, you cannot quite recover the original images. Run many times, and for many latent spaces, epochs and batch sizes. Summarize what you learned. 

In [ ]:
def plot_interpolations(encoder, decoder, data, n=10, figsize=(20, 4)):
    """
    Visualizes the interpolation between two images in the latent space.
    
    Parameters:
    - encoder: The encoder model.
    - decoder: The decoder model.
    - data: The dataset to use for sampling images.
    - n: Number of interpolation steps.
    - figsize: Size of the figure.
    """
    
    # select two random images from the dataset
    idx = np.random.choice(len(data), 2, replace=False)
    img1, img2 = data[idx]
    
    # encode the images to get their latent representations
    z1, z2 = encoder.predict(np.array([img1, img2]))
    
    # interpolate between the latent representations
    interpolations = np.zeros((n, z1.shape[-1]))
    for i in range(n):
        interpolations[i] = z1 + (z2 - z1) * i / (n - 1)
    
    # decode the interpolated representations
    reconstructions = decoder.predict(interpolations)

    # plotting
    plt.figure(figsize=figsize)
    for i, img in enumerate(reconstructions):
        ax = plt.subplot(2, n, i + 1)
        plt.imshow(img.reshape(28, 28), cmap='gray')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    
    # plot the original images
    for i, img in enumerate([img1, img2]):
        ax = plt.subplot(2, n, 11 + i*9)
        plt.imshow(img.reshape(28, 28), cmap='plasma')
        ax.get_xaxis().set_visible(False)
        ax.get_yaxis().set_visible(False)
    
    plt.show()

plot_interpolations(encoder, decoder, test_images)
